In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate
import pathlib
import pdal
import json

# Setup paths

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
initial_path = pathlib.Path(r'initial_data')
catchment_trimmed_path = pathlib.Path(r'catchment_trimmed_data')
destination_path = pathlib.Path(r'combined_data')

lidar_name = pathlib.Path(r'points.laz')

trimmed_dem_name = pathlib.Path(r'dem.nc')
trimmed_bathymetry_name = pathlib.Path(r'offshore_bathymetry')
trimmed_boundary_land_name = pathlib.Path(r'boundary_land')

# Load in background data

In [ ]:
boundary = geopandas.read_file(base_path/catchment_trimmed_path/trimmed_boundary_land_name)
bathymetry_trimmed = geopandas.read_file(base_path/catchment_trimmed_path/trimmed_bathymetry_name)
dem_trimmed = rioxarray.rioxarray.open_rasterio(base_path/catchment_trimmed_path/trimmed_dem_name, masked=True)

# Set to same projection

In [ ]:
crs = 2193
boundary = boundary.to_crs(crs)
bathymetry_trimmed = bathymetry_trimmed.to_crs(crs)
dem_trimmed.rio.set_crs(crs);

# Read in LiDAR with relevant processing
Load in the LAZ file with relevant processing
*  Set projection: https://pdal.io/stages/filters.reprojection.html#filters-reprojection
*  Crop within boundary: https://pdal.io/stages/filters.crop.html#filters-crop
*  Create a polygon of the LiDAR extents for filtering the DEM: https://pdal.io/stages/filters.hexbin.html#filters-hexbin

In [ ]:
lidar_file_name = base_path/initial_path/lidar_name

pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
    {"type":"filters.crop", "polygon":str(boundary.loc[0].geometry)}, # filter within boundary
    {"type" : "filters.hexbin"} # create a polygon boundary of the LiDAR
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

# Remove DEM where LiDAR
Will try get a polygon of hte DEM extents as part of the PDAL processing pipeline. Create a polygon of the LiDAR extents to then filter the DEM. It looks like this can be done with filters. hexbin. https://pdal.io/stages/filters.hexbin.html#filters-hexbin

In [ ]:
metadata=json.loads(pdal_pipeline.get_metadata())
lidar_boundary=shapely.wkt.loads(metadata['metadata']['filters.hexbin']['boundary'])

Filter all internal areas less than some percentage

In [ ]:
percentage_to_drop = 5
area_to_drop = shapely.geometry.Polygon(lidar_boundary.exterior).area * percentage_to_drop / 100
filtered_lidar_boundary = shapely.geometry.Polygon(lidar_boundary.exterior.coords,
            [interior for interior in lidar_boundary.interiors if shapely.geometry.Polygon(interior).area > area_to_drop])
filtered_lidar_boundary=geopandas.GeoSeries([filtered_lidar_boundary])

Filter DEM inside the LiDAR region

In [ ]:
dem_lidar_trimmed = dem_trimmed.rio.clip([filtered_lidar_boundary.loc[0]], invert=True)

# Plot the Lidar trimmed results

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

filtered_lidar_boundary.plot(ax=ax, color='None', edgecolor='teal', linewidth=5)
p = ax.pcolormesh(dem_lidar_trimmed.x[:],dem_lidar_trimmed.y[:],dem_lidar_trimmed.values[0],cmap='viridis', shading='auto')
boundary.plot(ax=ax, color='None', edgecolor='lime', linewidth=5)
bathymetry_trimmed.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.colorbar(p)

ax.set_title("Combined")

# Combine in single point data set
The data to combine is the:
* LiDAR
* DEM centroids
* Sounding values
Extract LiDAR array

In [ ]:
lidar_array = pdal_pipeline.arrays[0]

Extract valid DEM x, y, z pairs

In [ ]:
dem_x, dem_y = numpy.meshgrid(dem_lidar_trimmed.x, dem_lidar_trimmed.y)
dem_z = dem_lidar_trimmed.data[0].flatten()
dem_filtered_x = dem_x.flatten()[~numpy.isnan(dem_z)]
dem_filtered_y = dem_y.flatten()[~numpy.isnan(dem_z)]
dem_filtered_z = dem_z[~numpy.isnan(dem_z)]

Extract valid Bathymetry x, y, z pairs

In [ ]:
bathymetry_trimmed_x = bathymetry_trimmed.apply(lambda x : x['geometry'][0].x,axis=1).to_numpy()
bathymetry_trimmed_y = bathymetry_trimmed.apply(lambda x : x['geometry'][0].y,axis=1).to_numpy()
bathymetry_trimmed_z = bathymetry_trimmed.apply(lambda x : x['geometry'][0].z,axis=1).to_numpy() * -1 # map depth to elevatation

Combine values

In [ ]:
combined_x = numpy.concatenate([lidar_array['X'], dem_filtered_x, bathymetry_trimmed_x])
combined_y = numpy.concatenate([lidar_array['Y'], dem_filtered_y, bathymetry_trimmed_y])
combined_z = numpy.concatenate([lidar_array['Z'], dem_filtered_z, bathymetry_trimmed_z])

# View background trimmed data

In [ ]:
n_thining = 1

f = matplotlib.pyplot.scatter(combined_x[::n_thining], combined_y[::n_thining], c=combined_z[::n_thining], 
                          marker='.', cmap = 'viridis', vmin=-10, vmax=20)
matplotlib.pyplot.colorbar()
matplotlib.pyplot.title("combined filtered points")

# Save trimmed values
Append trimmed background points to the filteed LiDAR points

In [ ]:
background_x = numpy.concatenate([dem_filtered_x, bathymetry_trimmed_x])
background_y = numpy.concatenate([dem_filtered_y, bathymetry_trimmed_y])
background_z = numpy.concatenate([dem_filtered_z, bathymetry_trimmed_z])

background_lidar = numpy.zeros_like(lidar_array, shape=[len(background_x)])
background_lidar['X'] = background_x
background_lidar['Y'] = background_y
background_lidar['Z'] = background_z

combined_lidar_array = numpy.concatenate([lidar_array, background_lidar])

Save out the appended values as a LAZ file

In [ ]:
lidar_file_name = base_path/destination_path/lidar_name

pdal_pipeline_instructions = [
    {"type":  "writers.las", "filename": str(lidar_file_name), "a_srs": "EPSG:" + str(crs)},
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [combined_lidar_array])
pdal_pipeline.execute();